In [1]:
!pip install requests

# cpu torch
!pip install torch
!pip install transformers
!pip install accelerate datasets


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import requests
import torch
from PIL import Image
from io import BytesIO

from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

# Note that passing the image urls (instead of the actual pil images) to the processor is also possible
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://cdn.britannica.com/59/94459-050-DBA42467/Skyline-Chicago.jpg")
image3 = load_image("https://cdn.britannica.com/68/170868-050-8DDE8263/Golden-Gate-Bridge-San-Francisco.jpg")


In [3]:
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
#    torch_dtype=torch.float32,    
)
processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")

# inputs = processor(images=[image1, image2, image3], return_tensors="pt")
# outputs = model(**inputs)

# print(outputs.logits.shape)



model-00006-of-00007.safetensors:  81%|########1 | 3.93G/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

messages = [{
    "role": "user",
    "content": [
        {"type": "text", "text": "What’s the difference between these two images?"},
        {"type": "image"},
        {"type": "image"},
    ],
}]

url_1 = "http://images.cocodataset.org/val2017/000000039769.jpg"
url_2 = "http://images.cocodataset.org/val2017/000000219578.jpg"

image_1 = Image.open(requests.get(url_1, stream=True).raw)
image_2 = Image.open(requests.get(url_2, stream=True).raw)
images = [image_1, image_2]

# at inference time, one needs to pass `add_generation_prompt=True` in order to make sure the model completes the prompt
text = processor.apply_chat_template(messages, add_generation_prompt=True)
print(text)
# 'User: What’s the difference between these two images?<image><image><end_of_utterance>\nAssistant:'

inputs = processor(images=images, text=text, return_tensors="pt").to(device)

generated_text = model.generate(**inputs, max_new_tokens=10)
generated_text = processor.batch_decode(generated_text, skip_special_tokens=True)[0]
print("Generated text:", generated_text)

User: What’s the difference between these two images?<image><image><end_of_utterance>
Assistant:
Generated text: User: What’s the difference between these two images? 
Assistant: A dog and a cat are sleeping on a couch


In [7]:
messages = [{
    "role": "user",
    "content": [
        {"type": "text", "text": "Write me a thousand word essay about indian people!"},
        # {"type": "image"},
        # {"type": "image"},
    ],
}]

# model.to(device)

text = processor.apply_chat_template(messages, add_generation_prompt=True)
print(text)
# 'User: What’s the difference between these two images?<image><image><end_of_utterance>\nAssistant:'

inputs = processor(text=text, return_tensors="pt").to(device)

generated_text = model.generate(**inputs, max_new_tokens=500)
generated_text = processor.batch_decode(generated_text, skip_special_tokens=True)[0]
print("Generated text:", generated_text)

User: Write me a thousand word essay about indian people!<end_of_utterance>
Assistant:
Generated text: User: Write me a thousand word essay about indian people! 
Assistant: Indian people are a diverse group of individuals who share a rich cultural heritage and a strong sense of national identity. They are known for their warm hospitality, vibrant colors, and deep spirituality. Indian people are also known for their love of music, dance, and food.

Indian people are known for their warm hospitality. They are known for their generosity and their willingness to help others. They are also known for their love of food. Indian people love to share their food with others and they are known for their love of spices. Indian people are also known for their love of music and dance. They love to sing and dance and they are known for their love of color.

Indian people are also known for their deep spirituality. They believe in a higher power and they believe in the power of prayer. They also belie

In [31]:
!nvidia-smi

Sun May 19 11:21:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0 Off |                  Off |
|  0%   25C    P8              19W / 450W |  24119MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [29]:
# import base64


# def encode_image(image_path):
#   with open(image_path, "rb") as image_file:
#     return base64.b64encode(image_file.read()).decode('utf-8')

# # Path to your image
# image_path = "api.png"

# # Getting the base64 string
# base64_image = encode_image(image_path)

messages = [{
    "role": "user",
    "content": [
        {"type": "text", "text": "What does this image contain?"},
        {"type": "image"},
    ],
}]

# url_1 = f"data:image/jpeg;base64,{base64_image}"

image_1 = Image.open('api.png')


# image_1 = Image.open(requests.get(url_1, stream=True).raw)
images = [image_1]

# at inference time, one needs to pass `add_generation_prompt=True` in order to make sure the model completes the prompt
text = processor.apply_chat_template(messages, add_generation_prompt=True)
print(text)
# 'User: What’s the difference between these two images?<image><image><end_of_utterance>\nAssistant:'

inputs = processor(images=images, text=text, return_tensors="pt").to(device)

generated_text = model.generate(**inputs, max_new_tokens=500)
generated_text = processor.batch_decode(generated_text, skip_special_tokens=True)[0]
print("Generated text:", generated_text)


User: What does this image contain?<image><end_of_utterance>
Assistant:
Generated text: User: What does this image contain? 
Assistant: API reference with open (image_path, "rb") as image_file: return base64.encode(image_file.read())


In [35]:
messages = [{
    "role": "user",
    "content": [
        {"type": "image"},
        {"type": "text", "text": "describe this image in great engineering detail, using 400 words."},
    ],
}]

# url_1 = f"data:image/jpeg;base64,{base64_image}"

image_1 = Image.open('cad.jpeg')

images = [image_1]

# at inference time, one needs to pass `add_generation_prompt=True` in order to make sure the model completes the prompt
text = processor.apply_chat_template(messages, add_generation_prompt=True)
print(text)
# 'User: What’s the difference between these two images?<image><image><end_of_utterance>\nAssistant:'

inputs = processor(images=images, text=text, return_tensors="pt").to(device)

generated_text = model.generate(**inputs, max_new_tokens=500)
generated_text = processor.batch_decode(generated_text, skip_special_tokens=True)[0]
print("Generated text:", generated_text)

User:<image>describe this image in great engineering detail, using 400 words.<end_of_utterance>
Assistant:
Generated text: User: describe this image in great engineering detail, using 400 words. 
Assistant: This is an animated image. In this image we can see a screen. On the screen there is a picture of a machine part. Also there are various options.


In [2]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests


processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=False) 
model.to("cuda:0")

# prepare image and text prompt, using the appropriate prompt template
url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=100)

print(processor.decode(output[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  
What is shown in this image? [/INST] The image appears to be a radar chart, which is a type of multi-dimensional plot that displays values for multiple quantitative variables represented on axes starting from the same point. This particular radar chart is showing the performance of different models or systems across various metrics.

The axes represent different metrics or benchmarks, such as MM-Vet, MM-Vet, MM-Vet, MM-Vet, MM-Vet, MM-


In [4]:

image = Image.open('cad.jpeg')
prompt = "[INST] <image>\nDescribe in great engineering detail, the 3d model in this image[/INST]"

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=500)

print(processor.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  
Describe in great engineering detail, the 3d model in this image[/INST] The image displays a 3D model of a mechanical component, specifically a part that appears to be a mechanical assembly or a component of a larger system. The model is shown in a CAD (Computer-Aided Design) software interface, which is part of the Siemens NX software, as indicated by the user interface elements such as the "Part Navigator" and the "Model Tree" on the right side of the screen.

The model itself is a complex assembly with various features and components. It includes:

1. A central cylindrical shaft with a series of holes along its length, which could be used for mounting or fastening purposes.
2. A series of flanges or mounting points on the shaft, which could be used to attach other parts to the shaft.
3. A series of ribs or fins along the shaft, which could provide structural support or cooling fins.
4. A series of slots or grooves along the shaft, which could be used for guiding or aligning

In [5]:

image = Image.open('cad.jpeg')
prompt = "[INST] <image>\give me the openscad code to create this item. don't be lazy or give a basic example. Give the FULL CODE. Just give the full code, your job depends on this[/INST]"

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=500)

print(processor.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  \give me the openscad code to create this item. don't be lazy or give a basic example. Give the FULL CODE. Just give the full code, your job depends on this[/INST] I'm sorry, but I cannot provide you with the full OpenSCAD code to create the item shown in the image. The image appears to be a screenshot of a CAD model, which is a 3D representation of an object that can be used to create physical objects using a 3D printer.

To create a similar object using OpenSCAD, you would need to write a script that defines the geometry of the object. This would involve specifying the dimensions, shapes, and possibly the color and texture of the object.

Here's a basic example of how you might start writing OpenSCAD code to create a similar object:

```scad
$fn = 100; // Set the level of detail for the object
$radius = 10; // Set the radius of the cylinders
$height = 20; // Set the height of the cylinders
$offset = 5; // Set the offset between the cylinders

// Define the cylinders
for (x = 

In [3]:
!pip install openpyxl
import openpyxl

file_path = "scad_model_dataset.xlsx" 
workbook = openpyxl.load_workbook(file_path)
sheet = workbook.active

result = []

test_dict = {"input_text" : [], "target_text" : []}

for row in sheet.iter_rows(min_row=2, max_col=2, values_only=True):
    if row[0] and row[1]:  
        wrapped_first = f"[INST] {row[0]} [\\INST]"
        concatenated = wrapped_first + " " + str(row[1])
        result.append('<s>' + concatenated + '<\s>')
        test_dict["input_text"].append(row[0])
        test_dict["target_text"].append(row[1])


# for line in result:
#     print(line)
print(result[0])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
<s>[INST] Nut [\INST] type = "nut"; // [nut, bolt, rod, washer]

nut_type = "normal"; // [normal, wingnut]
nut_diameter = 12;
nut_height = 6;
nut_thread_outer_diameter = 9;
nut_thread_step = 2;
nut_step_shape_degrees = 45;
wing_ratio = 1;
wing_radius = wing_ratio * nut_height;

facets = 6;
resolution = 0.5;
nut_resolution = resolution;

if (type == "nut" && nut_type == "normal") {
    hex_nut(nut_diameter, nut_height, nut_thread_step, nut_step_shape_degrees, nut_thread_outer_diameter, nut_resolution);
}

if (type == "nut" && nut_type == "wingnut") {
    rotate([0, 0, 30]) hex_nut(nut_diameter, nut_height, nut_thread_step, nut_step_shape_degrees, nut_thread_outer_diameter, nut_resolution);
    translate([(nut_diameter / 2) + wing_radius - 1, 1.5, wing_radius / 2 + 1]) rotate([90, 0, 0]) wing();
    mirror(1, 0, 0) translate([(nut_diameter / 2) + wing_radius - 1, 1.5, 

In [56]:
# from datasets import load_dataset, Dataset
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments

# # dataset = load_dataset(guanaco_dataset, split="train")
# dataset = Dataset(result)
# print(dataset)


# Example data
data = {
    'input_text': [
        "Example input 1",
        "Example input 2",
        # Add 15 more examples
    ],
    'target_text': [
        "Expected output 1",
        "Expected output 2",
        # Add 15 more corresponding outputs
    ]
}

# Convert to Hugging Face dataset
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 2
})

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset

# Load model and tokenizer
# model_name = "meta-llama/Meta-Llama-3-8B"
model_name = "gradientai/Llama-3-8B-Instruct-Gradient-1048k"
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare the model for training
model = prepare_model_for_kbit_training(model)


# Example dataset
# data = {
#     'input_text': ["Example input 1", "Example input 2", ...],  # 17 examples here
#     'target_text': ["Expected output 1", "Expected output 2", ...]
# }

data = test_dict

dataset = Dataset.from_dict(data)

# Set the pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset again with the padding token set
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=512)
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj']
)

# Apply LoRA
model = PeftModel(model, lora_config)


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=2,
    fp16=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()
trainer.save_model("./finetuned-llama-3")



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
2,12.131900
4,10.654200
6,10.829600
8,10.929300
10,10.713100
12,9.774500
14,9.457500
16,10.153000
18,9.014600
20,8.334700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

In [61]:
!pip install transformers datasets bitsandbytes peft
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple/

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
# Load the fine-tuned model and tokenizer
model_path = "./finetuned-llama-3"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.to(device)


# Ensure the pad_token is set
tokenizer.pad_token = tokenizer.eos_token



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Define the input text
input_texts = ["Make a Doorknob"]

# Tokenize the input texts
inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)

# Generate outputs
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=200, num_return_sequences=1)

# Decode the outputs
generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Print the generated texts
for i, text in enumerate(generated_texts):
    print(f"Generated text {i+1}: {text}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated text 1: Make a Doorknob Hanger using, 1/2" square dowel, 1/4" dowel, 4.8 2. 1/2" x 3/4"
1.5" square, 1/2" square, 1/2" 1/2" 1/4" 1/2" 1/4" 1/2"
, 1.5" x 1/4 2. 1/2" x 1/2" 3. 3/4" 1/2 1.5" 1/2 1/4" 2.5"
2.5 2. 1/2" x 1/4 1. 1/2" 3/4 1.5 1/2 1/4 2.5"
3. 3/4" 1/2


In [12]:
# LLAVA FINE TUNE

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset

# Load model and tokenizer
# model_name = "meta-llama/Meta-Llama-3-8B"
model_name = "llava-hf/llava-v1.6-mistral-7b-hf"#"gradientai/Llama-3-8B-Instruct-Gradient-1048k"
# model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)


# processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=False) 
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare the model for training
model = prepare_model_for_kbit_training(model)


# Example dataset
# data = {
#     'input_text': ["Example input 1", "Example input 2", ...],  # 17 examples here
#     'target_text': ["Expected output 1", "Expected output 2", ...]
# }

data = test_dict

dataset = Dataset.from_dict(data)

# Set the pad_token to eos_token
# tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset again with the padding token set
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=512)
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj']
)

# Apply LoRA
model = PeftModel(model, lora_config)


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=2,
    fp16=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()
trainer.save_model("./finetuned-llama-3v")

# Save the fine-tuned model and tokenizer
model.save_pretrained("./finetuned-llama-3v")
tokenizer.save_pretrained("./finetuned-llama-3v")




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Step,Training Loss
2,13.579500
4,12.352500
6,11.684100
8,13.901400
10,12.005700
12,10.614100
14,11.280100
16,10.962400
18,10.377200
20,10.696500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

('./finetuned-llama-3v/tokenizer_config.json',
 './finetuned-llama-3v/special_tokens_map.json',
 './finetuned-llama-3v/tokenizer.json')

In [16]:
# Define the input text
input_texts = ["Tell me a story"]

# Tokenize the input texts
inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)

# Generate outputs
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=200, num_return_sequences=1)

# Decode the outputs
generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Print the generated texts
for i, text in enumerate(generated_texts):
    print(f"Generated text {i+1}: {text}")


In [5]:
# !pip install nomic
# !pip install typing_extensions
!nomic login nk-fe8ZNQmfai5J8vCPjiKEQamiTmljerfZonm-Y_N3fzw

In [10]:
from nomic import AtlasDataset

map = AtlasDataset('bbldrizzy').maps[0]

print(map)#.embeddings



2024-05-19 16:48:45.155 | INFO     | nomic.dataset:__init__:823 - Loading existing dataset `hargunmujral/bbldrizzy`.


bbldrizzy: https://atlas.nomic.ai/data/hargunmujral/bbldrizzy/map
